# SparkContext and RDD basics



Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

### Import libraries

In [24]:
!pip install pyspark
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
from pyspark import SparkContext
import numpy

## Initialize a `SparkContext` (the main abstraction to the cluster)
**Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.**

In [6]:
sc=SparkContext(master="local[4]")

22/10/11 12:58:51 WARN Utils: Your hostname, tars resolves to a loopback address: 127.0.1.1; using 192.168.18.24 instead (on interface wlan0)
22/10/11 12:58:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 12:58:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


### Generate a list of random integeres

In [8]:
lst=np.random.randint(0,10,20)

In [9]:
print(lst)

[4 7 2 6 2 3 4 0 6 4 7 3 7 9 1 7 3 1 3 6]


### Parallelize the list - this is the main operation toward distributed computing

In [10]:
A=sc.parallelize(lst)

### What did we just do? We created a RDD? What is a RDD?
![](https://i.stack.imgur.com/cwrMN.png)
Spark revolves around the concept of a resilient distributed dataset (RDD), which is a **fault-tolerant collection of elements that can be operated on in parallel**. SparkContext manages the distributed data over the worker nodes through the cluster manager. 

There are two ways to create RDDs: 
* parallelizing an existing collection in your driver program, or 
* referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

We created a RDD using the former approach

### `A` is a pyspark RDD object, we cannot access the elements directly

In [11]:
type(A)

pyspark.rdd.RDD

In [12]:
A

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

### Opposite to parallelization - `collect` brings all the distributed elements and returns them to the head node. <br><br>Note - this is a slow process, do not use it often. 

In [13]:
A.collect()

[4, 7, 2, 6, 2, 3, 4, 0, 6, 4, 7, 3, 7, 9, 1, 7, 3, 1, 3, 6]

### How were the partitions created? Use `glom` method

In [26]:
A.glom().collect()

22/10/11 18:36:21 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 4)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 1055, in func
    yield list(iterator)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 151, in load_stream
    yield self._read_with_length(stream)
  File "/home/tars/.local/lib/python3.10/site-packages

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 4) (192.168.18.24 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 1055, in func
    yield list(iterator)
  File "/h

### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [19]:
sc.stop()

In [20]:
sc=SparkContext(master="local[2]")

In [21]:
A = sc.parallelize(lst)

In [22]:
A.glom().collect()

22/10/11 18:34:23 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 1055, in func
    yield list(iterator)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 151, in load_stream
    yield self._read_with_length(stream)
  File "/home/tars/.local/lib/python3.10/site-packages

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.18.24 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/tars/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 1055, in func
    yield list(iterator)
  File "/h

**The RDD is now distributed over two chunks, not four!** 

So, let's redo the process with 4 cores again.

In [22]:
sc.stop()

In [23]:
sc = SparkContext(master="local[4]")

In [24]:
A = sc.parallelize(lst)

## Basic operations
### `Count` the elements

In [25]:
A.count()

20

### The first element (`first`) and the first few elements (`take`)

In [26]:
A.first()

5

In [27]:
A.take(4)

[5, 9, 2, 8]

### Removing duplicates: Get another RDD with only the `distinct` elements

The method `RDD.distinct()` Returns a new dataset that contains the distinct elements of the source dataset.

**NOTE**: This operation requires a **shuffle** in order to detect duplication across partitions. **So, it is a slow operation.**

In [28]:
A_distinct=A.distinct()

In [29]:
A_distinct.collect()

[8, 4, 5, 9, 1, 2, 6, 7, 3]

### To sum all the elements use `reduce` method

In [30]:
A.reduce(lambda x,y:x+y)

106

### Or direct `sum` method

In [31]:
A.sum()

106

### Or using the `fold` method, which aggregates the elements of each partition, and then the results for all the partitions

In [25]:
A.fold(0,lambda x,y:x+y)

80

### Finding maximum element by `reduce`

In [32]:
A.reduce(lambda x,y: x if x > y else y)

9

### Finding longest word using `reduce`

In [33]:
words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'computers'

## Functions/filtering over RDD
### Use `filter` to return a new RDD with elements satisfying a given predicate (lambda expression)

In [34]:
# Return RDD with elements divisible by 3
A.filter(lambda x:x%3==0).collect()

[9, 9, 6, 6, 3, 9, 6]

### Lambda functions are short and sweet but we can write regular Python functions to use with `reduce`

In [35]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y

In [36]:
wordRDD.reduce(largerThan)

'Macintosh'

## Sampling an RDD
* RDDs are often very large.
* **Aggregates, such as averages, can be approximated efficiently by using a sample.** This comes handy often for operation with extremely large datasets where a sample can tell a lot about the pattern and descriptive statistics of the data.
* Sampling is done in parallel and requires limited computation.

The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [40]:
# get a sample whose expected size is m
# Note that the size of the sample is different in different runs
m=5
n=20
print('sample1=',A.sample(False,m/n).collect()) 
print('sample2=',A.sample(False,m/n).collect())
print('sample3=',A.sample(False,m/n).collect())
print('sample4=',A.sample(False,m/n).collect())

sample1= [2, 8, 8, 1, 9, 5]
sample2= [7, 9, 6, 3, 1]
sample3= [9, 8, 9, 1, 6, 5]
sample4= [9, 2, 6, 6, 1]


### Things to note and think about
* Each time you run the previous cell, you get a different estimate
* The accuracy of the estimate is determined by the size of the sample $n*p$. Here, probability $p=\frac{m}{n}$
* See how the error changes as you vary $p$

## Basic statistics

In [41]:
print("Maximum: ",A.max())
print("Minimum: ",A.min())
print("Mean (average): ",A.mean())
print("Standard deviation: ",A.stdev())

Maximum:  9
Minimum:  1
Mean (average):  5.3
Standard deviation:  2.8478061731796283


In [33]:
A.stats()

(count: 20, mean: 4.0, stdev: 2.9832867780352594, max: 9.0, min: 0.0)

## Mapping
### `map` operation with _lambda_ function

In [34]:
B=A.map(lambda x:x*x)

In [35]:
B.collect()

[16, 64, 4, 4, 16, 49, 0, 9, 9, 81, 4, 36, 0, 0, 1, 49, 25, 1, 81, 49]

### `map` operation with regular Python function

In [42]:
def square_if_odd(x):
    if x%2==1:
        return x*x
    else:
        return x

In [43]:
A.map(square_if_odd).collect()

[25, 81, 2, 8, 49, 8, 4, 8, 1, 81, 1, 1, 6, 6, 9, 81, 6, 1, 49, 25]

### `flatmap` method returns a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [44]:
A.flatMap(lambda x:(x,x*x)).collect()

[5,
 25,
 9,
 81,
 2,
 4,
 8,
 64,
 7,
 49,
 8,
 64,
 4,
 16,
 8,
 64,
 1,
 1,
 9,
 81,
 1,
 1,
 1,
 1,
 6,
 36,
 6,
 36,
 3,
 9,
 9,
 81,
 6,
 36,
 1,
 1,
 7,
 49,
 5,
 25]

## Grouping and binning
### `groupby` returns a RDD of grouped elements (iterable) as per a given group operation (function)

In [52]:
result=A.groupBy(lambda x:x%2).collect()
print(A.collect())
#print(sorted(result[0][1]))
sorted([(x, sorted(y)) for (x, y) in result])

[5, 9, 2, 8, 7, 8, 4, 8, 1, 9, 1, 1, 6, 6, 3, 9, 6, 1, 7, 5]


[(0, [2, 4, 6, 6, 6, 8, 8, 8]), (1, [1, 1, 1, 1, 3, 5, 5, 7, 7, 9, 9, 9])]

### `histogram` method takes a list of bins/buckets and returns a tuple with result of the histogram (binning) 

In [54]:
A.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [20, 0, 0, 0, 0, 0, 0, 0, 0])

## Set operations
### Create smaller RDDs to demonstrate joint operations

In [56]:
lst1=np.random.randint(0,10,3)
C=sc.parallelize(lst1)
lst2=np.random.randint(10,20,3)
D=sc.parallelize(lst2)
print("C:",C.collect())
print("D:",D.collect())

C: [4, 0, 2]
D: [13, 14, 11]


### `C+D` gives the union (like set union), not the element wise sum

In [57]:
(C+D).collect()

[4, 0, 2, 13, 14, 11]

### `cartesian` gives the pairwise product (as tuples) 

In [58]:
C.cartesian(D).collect()

[(4, 13),
 (4, 14),
 (4, 11),
 (0, 13),
 (0, 14),
 (0, 11),
 (2, 13),
 (2, 14),
 (2, 11)]

### `intersection` and `subtract `methods return a RDD of the set intersection and subtraction (difference)

In [59]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

In [60]:
rdd1.subtract(rdd2).collect()

[10, 4, 5]

### Stop the `SparkContext` at the end

In [61]:
sc.stop()